In [1]:
import os
os.chdir("../")

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import RobustScaler
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, InputLayer, GaussianNoise
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import train_test_split
import tensorflow as tf

Using TensorFlow backend.


In [3]:
os.getcwd()

'f:\\project3'

In [36]:
from preprocessing.Normalize import Normalize
import helper.SeriesHelper as series_helper

In [37]:
normal_matrix = Normalize().get_normalized_data()
X = normal_matrix.to_numpy()
y = series_helper.get_relapse_value_from_series_matrix(normal_matrix)

In [38]:
# Sscaling features
scaler = RobustScaler()
X = scaler.fit_transform(X)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33)

In [7]:
# Keep only the most contributing features
# sfm = SelectFromModel(LassoCV())
# sfm.fit(X_train, y_train)
# X_train = sfm.transform(X_train)
# X_test = sfm.transform(X_test)

In [8]:
# pd.DataFrame(X_train).to_csv("X_train.csv")
# pd.DataFrame(X_test).to_csv("X_test.csv")
# pd.DataFrame(y_train).to_csv("y_train.csv")
# pd.DataFrame(y_test).to_csv("y_test.csv")

In [4]:
X_train = pd.read_csv("X_train.csv")
X_test = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv")
y_test = pd.read_csv("y_test.csv")

In [40]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true - y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [41]:

def build_model_fn(neurons=50, noise=0.25):
    model = Sequential()
    model.add(InputLayer(input_shape=(X_train.shape[1],)))
    model.add(GaussianNoise(noise))
    model.add(Dense(neurons, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=[r2_keras])
    return model

In [42]:
#
# Tuning model parameters
#
model = KerasRegressor(build_fn=build_model_fn, epochs=100, verbose=0)

In [43]:
gsc = GridSearchCV(
    estimator=model,
    param_grid={
        #'neurons': range(18,31,4),
        'noise': [x/20.0 for x in range(3, 7)],
    },
    #scoring='r2',
    scoring='neg_mean_squared_error',
    cv=5
)

In [44]:
grid_result = gsc.fit(X_train, y_train)

C:\Users\subas\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [49]:
grid_result.cv_results_

{'mean_fit_time': array([140.4731916 , 155.21334977, 150.54983373, 133.06099405]),
 'std_fit_time': array([15.67006872,  5.3825702 , 37.16268435,  6.92313102]),
 'mean_score_time': array([1.27036304, 2.37593155, 2.03602762, 1.90557199]),
 'std_score_time': array([0.49081274, 0.4996602 , 0.3884246 , 0.34055637]),
 'param_noise': masked_array(data=[0.15, 0.2, 0.25, 0.3],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'noise': 0.15}, {'noise': 0.2}, {'noise': 0.25}, {'noise': 0.3}],
 'split0_test_score': array([-0.29026788, -0.25012226, -0.57763975, -0.55603381]),
 'split1_test_score': array([-0.50625, -0.325  , -0.44375, -0.5125 ]),
 'split2_test_score': array([-0.5482556 , -0.5875    , -0.39138341, -0.33125   ]),
 'split3_test_score': array([-0.45970145, -0.50625   , -0.46875   , -0.39375   ]),
 'split4_test_score': array([-0.4    , -0.4    , -0.31875, -0.38125]),
 'mean_test_score': array([-0.44070694, -0.41357014, -0.4

In [50]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
for test_mean, test_stdev, train_mean, train_stdev, param in zip(
        grid_result.cv_results_['mean_test_score'],
        grid_result.cv_results_['std_test_score'],
        grid_result.cv_results_['mean_train_score'],
        grid_result.cv_results_['std_train_score'],
        grid_result.cv_results_['params']):
    print("Train: %f (%f) // Test : %f (%f) with: %r" % (train_mean, train_stdev, test_mean, test_stdev, param))
    

Best: -0.413570 using {'noise': 0.2}


KeyError: 'mean_train_score'

In [51]:
#
# Train model with best params for submission
#
model = build_model_fn(**grid_result.best_params_)

In [52]:
model.fit(X_train, y_train, epochs=500, verbose=2)

poch 162/500
 - 2s - loss: 6.9791 - r2_keras: -1.2486e+04
Epoch 163/500
 - 2s - loss: 6.9791 - r2_keras: -1.0266e+00
Epoch 164/500
 - 2s - loss: 6.9791 - r2_keras: -1.0622e+00
Epoch 165/500
 - 1s - loss: 6.9791 - r2_keras: -1.0769e+00
Epoch 166/500
 - 2s - loss: 6.9791 - r2_keras: -1.2485e+04
Epoch 167/500
 - 2s - loss: 6.9791 - r2_keras: -1.0110e+00
Epoch 168/500
 - 2s - loss: 6.9791 - r2_keras: -1.0454e+00
Epoch 169/500
 - 2s - loss: 6.9791 - r2_keras: -1.0467e+00
Epoch 170/500
 - 2s - loss: 6.9791 - r2_keras: -1.0463e+00
Epoch 171/500
 - 2s - loss: 6.9791 - r2_keras: -1.0030e+00
Epoch 172/500
 - 1s - loss: 6.9791 - r2_keras: -1.0474e+00
Epoch 173/500
 - 2s - loss: 6.9791 - r2_keras: -1.2485e+04
Epoch 174/500
 - 2s - loss: 6.9791 - r2_keras: -1.2485e+04
Epoch 175/500
 - 2s - loss: 6.9791 - r2_keras: -1.2485e+04
Epoch 176/500
 - 3s - loss: 6.9791 - r2_keras: -1.0239e+00
Epoch 177/500
 - 3s - loss: 6.9791 - r2_keras: -1.2485e+04
Epoch 178/500
 - 2s - loss: 6.9791 - r2_keras: -1.2485e+0

In [53]:
y_pred = model.predict(X_test).flatten()

In [54]:
y_pred.shape

(396,)

In [64]:
y_pred = y_pred.astype(int)

In [67]:
from sklearn.metrics import accuracy_score

In [68]:
accuracy_score(y_test, y_pred)

0.5707070707070707